In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget

In [3]:
import pandas as pd
import os
from pathlib import Path
import wandb

os.chdir('/home/bohdan/studies/sem8/io')
# os.chdir('/home/ehevi/studies/io-aco/')

In [4]:
csv_path = Path('small.csv')
problem = pd.read_csv(csv_path)
problem.head()


,cust,x,y,demand,from,to,service
0,1,40.0,50.0,0.0,0.0,1236.0,0.0
1,2,45.0,68.0,10.0,912.0,967.0,90.0
2,3,45.0,70.0,30.0,825.0,870.0,90.0
3,4,42.0,66.0,10.0,65.0,146.0,90.0
4,5,42.0,68.0,10.0,727.0,782.0,90.0


1. Uruchomić
 - dostosowanie do naszych danych
 
2. Dodać modyfikacje
- dodajemy do mrówki wiedzę o depocie oraz o pojemności
- mrówka po osiągnięciu maks pojemności wraca do początku



In [7]:
import random
from swarmlib_.aco4tsp.aco_algorithm import ACOAlgorithm
from swarmlib_.aco4tsp.vehicle import Vehicle
import numpy as np
from itertools import product

def evaluate_scenario(path: Path):
    vehicles = [Vehicle(50, 1) for _ in range(10)]
    config = {
        'ant_number': 5,
        'num_vehicles': 10,
        'iteration_number': 500,
        'capacity': 50,
        'commit_freq': 10,
        'vehicles': vehicles,
        'instance_path': 'small.csv',
        'rho': 0.5,
        'alpha': 0.5,
        'beta': 0.5,
    }

    
    random.seed(42)
    for alpha, rho, beta in product(np.arange(0.1, 1, 0.3), np.arange(0.1, 1, 0.3), np.arange(0.1, 1, 0.3)):
        wandb.init(
            project="dynamic aco",
            config={
                **config,
                'rho': rho,
                'alpha': alpha,
                'beta': beta,
            },
            tags=[f'04-06-2023-{path.stem}']
        )
        # Logic goes here
        try:
            solver = ACOAlgorithm(**config, scenario_input_file=path)
            best_plan, _ = solver.solve()
        finally:
            wandb.finish()


In [8]:
for scenario in ['scenario_mix.csv', 'scenario_decrease.csv']:
    evaluate_scenario(Path(scenario))

Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Iteration:  67
Iter

attractiveness_dispersion,▁▅▅▆▆▆▆▇▆▇▇▆▇▆▇▇▆▆▆▆▆▆▇▆▆▇▆▇▆▆▆▆▇▇▆█▇█▆▇
attractiveness_ratio,▇▅▁▃▃▅███▆▆▆▇▃▇▇▆▆▆▃▆▆▃▆▇█▅▆▆▇▇▄▆▆▅▃▇▇▄▅
best_fitness,█▆▆▆▅▅▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fitness,█▆█▇▇▅▃▄▄▅▄▄▃▅▃▁▅▅▃▄▅▄▅▄▄▃▆▃▄▄▃▅▅▅▆▄▂▄▅▄
overall_demand,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████
pheromone_ratio,▂▁▄▄▅▄▅▅▆▅▆▆▄▆▅▅▆█▇▆▆▆▆███▆▇▅██▇▇▅▆▅▆▆█▇
attractiveness_dispersion,4.59756
attractiveness_ratio,-1.25006
best_fitness,557.20955
fitness,687.20087
overall_demand,630.0


KeyboardInterrupt: 